# External Datasets

In this notebook, we assess the quality of two existing built-up datasets based on Landsat: the Global Human Settlements Layer (GHSL) and the Human Built-up and Settlements Extents (HBASE). The assessment is performed based on the reference hand-digitized land cover polygons located in `data/input/reference`. Three metrics are computed for each of our ten case studies: F1-Score, Precision & Recall.

## Imports & Parameters

In [1]:
import os, sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
import rasterio.mask
from tqdm import tqdm_notebook as tqdm
from shapely.geometry import shape
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
# Add local module to the path
src = os.path.abspath('../src')
if src not in sys.path:
    sys.path.append(src)

In [3]:
from metadata import City, CITIES, DATA_DIR
import classification as cls
from generate_aoi import reproject_geom, as_geojson

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Global Human Settlements Layer (GHSL)

The Global Human Settlements Layer Built-up Grid has been acquired through its [official website](http://ghsl.jrc.ec.europa.eu/ghs_bu.php).

In [7]:
ghsl_f = os.path.join(DATA_DIR, 'input', 'ghsl', 'GHS_BUILT_LDSMT_GLOBE_R2015B_3857_38_v1_0.vrt')

In [56]:
def get_ghsl(ghsl_f, area_of_interest, rasterio_profile):
    """Load GHSL data for a given area of interest (GeoJSON-like dict, EPSG:3857)."""
    masking_shapes = [area_of_interest['geometry']]
    
    with rasterio.open(ghsl_f) as src:
        img, src_affine = rasterio.mask.mask(src, masking_shapes, crop=True)
        img = img[0, :, :]
        src_crs = src.crs
    
    shape = (rasterio_profile['height'], rasterio_profile['width'])
    dst_affine, dst_crs = rasterio_profile['affine'], rasterio_profile['crs']
    layer = np.zeros(shape=shape, dtype=np.uint8)
    
    rasterio.warp.reproject(
        source=img,
        destination=layer,
        src_transform=src_affine,
        src_crs=src_crs,
        dst_transform=dst_affine,
        dst_crs=dst_crs,
    )
    
    return layer

In [21]:
METRICS = ['f1_score', 'precision', 'recall']
ghsl_scores = pd.DataFrame(index=CITIES, columns=METRICS)
progress = tqdm(total=len(CITIES))

for city_name in CITIES:
    
    city = City(city_name)

    area_of_interest = shape(city.aoi)
    area_of_interest = reproject_geom(area_of_interest, city.epsg, 3857)
    area_of_interest = as_geojson(area_of_interest)
    
    ghsl = get_ghsl(ghsl_f, area_of_interest, city.profile)
    
    y_true, y_pred = cls.transform_test(city.reference, ghsl)
    y_true = y_true == 1
    y_pred = y_pred >= 3
    
    ghsl_scores.at[(city_name, 'f1_score')] = f1_score(y_true, y_pred)
    ghsl_scores.at[(city_name, 'precision')] = precision_score(y_true, y_pred)
    ghsl_scores.at[(city_name, 'recall')] = recall_score(y_true, y_pred)
    progress.update(1)

progress.close()

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [54]:
ghsl_scores = ghsl_scores.astype(np.float64).round(2)
ghsl_scores.to_csv(os.path.join(DATA_DIR, 'output', 'ghsl_scores.csv'))
ghsl_scores

,f1_score,precision,recall
antananarivo,0.83,0.82,0.83
chimoio,0.47,0.97,0.31
dakar,0.85,0.74,1.00
gao,0.35,0.98,0.21
johannesburg,0.92,0.86,0.99
kampala,0.96,0.95,0.96
katsina,0.90,0.92,0.88
nairobi,0.84,0.96,0.75
saint_louis,0.76,0.95,0.63
windhoek,0.81,0.92,0.73


In [59]:
display(ghsl_scores.mean())
display(ghsl_scores.std())

f1_score     0.769
precision    0.907
recall       0.729
dtype: float64

f1_score     0.199580
precision    0.077323
recall       0.275498
dtype: float64

## Global Human Built-up and Settlement Extent (HBASE)

The HBASE data has been download manually through its [official website](http://sedac.ciesin.columbia.edu/data/set/ulandsat-hbase-v1/data-download) for each of our areas of interest.

In [25]:
hbase_dir = os.path.join(DATA_DIR, 'input', 'hbase')
hbase_f = '*_hbase_human_built_up_and_settlement_extent_utm_30m.tif'

In [34]:
def get_hbase(hbase_dir, hbase_f, city_name, dst_profile):
    """Load HBASE data for a given city and reproject it to the same CRS & affine.
    Some cities require two tiles in different CRS to be entirely covered. In this
    case, both tiles are merged.
    """
    dst_affine, dst_crs = dst_profile['affine'], dst_profile['crs']
    dst_shape = (dst_profile['height'], dst_profile['width'])
    
    hbase_files = glob(os.path.join(hbase_dir, city_name, hbase_f))    
    
    if len(hbase_files) == 1:
        
        path = hbase_files[0]
        
        with rasterio.open(path) as src:
            img = src.read(1)
            src_affine = src.affine
            src_crs = src.crs
        
        hbase = np.zeros(shape=dst_shape, dtype=np.uint8)
        rasterio.warp.reproject(
            source=img,
            destination=hbase,
            src_transform=src_affine,
            src_crs=src_crs,
            dst_transform=dst_affine,
            dst_crs=dst_crs
        )
        
        return hbase
        
    if len(hbase_files) == 2:
        
        path_a, path_b = hbase_files
        
        with rasterio.open(path_a) as src:
            img = src.read(1)
            src_affine = src.affine
            src_crs = src.crs
        
        hbase_a = np.zeros(shape=dst_shape, dtype=np.uint8)
        rasterio.warp.reproject(
            source=img,
            destination=hbase_a,
            src_transform=src_affine,
            src_crs=src_crs,
            dst_transform=dst_affine,
            dst_crs=dst_crs
        )

        with rasterio.open(path_b) as src:
            img = src.read(1)
            src_affine = src.affine
            src_crs = src.crs
        
        hbase_b = np.zeros(shape=dst_shape, dtype=np.uint8)
        rasterio.warp.reproject(
            source=img,
            destination=hbase_b,
            src_transform=src_affine,
            src_crs=src_crs,
            dst_transform=dst_affine,
            dst_crs=dst_crs
        )
        
        hbase_a[hbase_a < 200] = hbase_b[hbase_a < 200]
        return hbase_a

In [35]:
METRICS = ['f1_score', 'precision', 'recall']
hbase_scores = pd.DataFrame(index=CITIES, columns=METRICS)
progress = tqdm(total=len(CITIES))

for city_name in CITIES:
    
    city = City(city_name)
    
    hbase = get_hbase(hbase_dir, hbase_f, city.name, city.profile)
    
    y_true, y_pred = cls.transform_test(city.reference, hbase)
    y_true = y_true == 1
    y_pred = y_pred == 201
    
    hbase_scores.at[(city_name, 'f1_score')] = f1_score(y_true, y_pred)
    hbase_scores.at[(city_name, 'precision')] = precision_score(y_true, y_pred)
    hbase_scores.at[(city_name, 'recall')] = recall_score(y_true, y_pred)
    progress.update(1)

progress.close()

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

In [53]:
hbase_scores = hbase_scores.astype(np.float64).round(2)
hbase_scores.to_csv(os.path.join(DATA_DIR, 'output', 'hbase_scores.csv'))
hbase_scores

,f1_score,precision,recall
antananarivo,0.79,0.67,0.96
chimoio,0.82,0.94,0.73
dakar,0.81,0.69,0.98
gao,0.73,0.94,0.59
johannesburg,0.90,0.82,1.00
kampala,0.95,0.93,0.97
katsina,0.65,0.76,0.56
nairobi,0.88,0.81,0.97
saint_louis,0.81,0.97,0.70
windhoek,0.78,0.65,0.99


In [60]:
display(hbase_scores.mean())
display(hbase_scores.std())

f1_score     0.812
precision    0.818
recall       0.845
dtype: float64

f1_score     0.085609
precision    0.122638
recall       0.178963
dtype: float64

## Summary

In [10]:
from itertools import product

In [9]:
ghsl_scores = pd.read_csv(os.path.join(DATA_DIR, 'output', 'ghsl_scores.csv'), index_col=0)
hbase_scores = pd.read_csv(os.path.join(DATA_DIR, 'output', 'hbase_scores.csv'), index_col=0)

In [17]:
map_labels = ['ghsl', 'hbase']
metrics = ['f1_score', 'precision', 'recall']
columns = [label + '_' + metric for label, metric in product(map_labels, metrics)]
scores = pd.DataFrame(index=CITIES, columns=columns)

In [18]:
for map_scores, map_label in zip([ghsl_scores, hbase_scores], map_labels):
    
    for city, metric in product(CITIES, metrics):
        
        col = map_label + '_' + metric
        scores.at[(city, col)] = map_scores.at[(city, metric)]

In [19]:
scores.loc['mean'] = scores.mean().round(2)
scores.loc['std'] = scores.std().round(2)

In [20]:
scores

,ghsl_f1_score,ghsl_precision,ghsl_recall,hbase_f1_score,hbase_precision,hbase_recall
antananarivo,0.83,0.82,0.83,0.79,0.67,0.96
chimoio,0.47,0.97,0.31,0.82,0.94,0.73
dakar,0.85,0.74,1,0.81,0.69,0.98
gao,0.35,0.98,0.21,0.73,0.94,0.59
johannesburg,0.92,0.86,0.99,0.9,0.82,1
kampala,0.96,0.95,0.96,0.95,0.93,0.97
katsina,0.9,0.92,0.88,0.65,0.76,0.56
nairobi,0.84,0.96,0.75,0.88,0.81,0.97
saint_louis,0.76,0.95,0.63,0.81,0.97,0.7
windhoek,0.81,0.92,0.73,0.78,0.65,0.99
